<a href="https://colab.research.google.com/github/Hellork780/SEO-Machine-Learning-Models/blob/main/SEO_Keyword_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keyword Research Automation


✅ pytrends – Google Trends API
✅ SERPAPI – Google search scraping
✅ Google Ads API

In [3]:
!pip install pytrends



Find Trending Keywords

In [4]:
from pytrends.request import TrendReq
import pandas as pd

# Function to get Google Trends data dynamically
def get_trends(keywords, country, timeframe):
    pytrends = TrendReq(hl='en-US', tz=360)

    # Build payload with dynamic parameters
    pytrends.build_payload(kw_list=keywords, timeframe=timeframe, geo=country)

    # Get interest over time
    trends = pytrends.interest_over_time()

    # Check if data is retrieved
    if trends.empty:
        print("No data found. Try different parameters.")
    else:
        print(trends)

    return trends

# User input for keywords, country, and timeframe
keywords = ["Data Science", "Keyword Research", "SEO Automation","Machine Learning","AI in SEO"]  # Modify this list
country = "IN"  # Change country code ('IN' for India, 'US' for USA, etc.)
timeframe = "today 3-m"  # Modify timeframe ('now 7-d', 'today 12-m', etc.)

# Call function
trends_data = get_trends(keywords, country, timeframe)


            Data Science  Keyword Research  SEO Automation  Machine Learning  \
date                                                                           
2024-12-29            51                 2               0                54   
2024-12-30            57                 3               0                66   
2024-12-31            55                 2               0                53   
2025-01-01            54                 0               0                50   
2025-01-02            65                 4               0                72   
...                  ...               ...             ...               ...   
2025-03-25            71                 3               0                78   
2025-03-26            78                 3               0                81   
2025-03-27            71                 3               0                73   
2025-03-28            66                 3               0                65   
2025-03-29            53                

/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


Download and save Data

In [ ]:
# Save data to CSV file
trends_data.to_csv("google_trends_data.csv", index=True)
print("✅ Data saved as 'google_trends_data.csv'")

from google.colab import files

# Download the CSV file
files.download("google_trends_data.csv")


✅ Data saved as 'google_trends_data.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Trends Graph**

In [ ]:
import pandas as pd
import plotly.express as px
from pytrends.request import TrendReq

# Fetch Google Trends Data
def get_trends(keywords, country, timeframe):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(kw_list=keywords, timeframe=timeframe, geo=country)

    trends = pytrends.interest_over_time()

    if trends.empty:
        print("No data found. Try different parameters.")
    else:
        trends = trends.drop(columns=["isPartial"])  # Remove unnecessary column

    return trends

# Modify these values
keywords = ["Data Science", "Machine learning in SEO", "SEO Automation"]
country = "IN"
timeframe = "today 12-m"

# Get data
trends_data = get_trends(keywords, country, timeframe)
trends_data.reset_index(inplace=True)  # Reset index to use Date column

# Plot Interactive Graph
fig = px.line(trends_data, x="date", y=trends_data.columns[1:],
              title="📊 Google Trends Data Over Time",
              labels={"date": "Date", "value": "Search Interest"},
              markers=True)

# Customize Layout
fig.update_layout(
    hovermode="x unified",
    template="plotly_dark",
    xaxis_title="Date",
    yaxis_title="Search Interest",
    legend_title="Keywords",
)

# Show interactive chart
fig.show()


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



## Future Keyword Prediction



### Single-Keyword Comparison

In [8]:
!pip install prophet plotly

from pytrends.request import TrendReq
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go  # Import Plotly for interactive plots

# Function to get Google Trends data dynamically
def get_trends(keywords, country, timeframe):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(kw_list=keywords, timeframe=timeframe, geo=country)
    trends = pytrends.interest_over_time()

    if trends.empty:
        print("No data found. Try different parameters.")
    return trends

# User input for keywords, country, and timeframe
keywords = ["Data Science"]  # Predict for one keyword at a time
country = "IN"
timeframe = "today 12-m"  # Get past 12 months of data

# Get trends data
trends_data = get_trends(keywords, country, timeframe)

# Prepare data for Prophet
if not trends_data.empty:
    df = trends_data.reset_index()[["date", keywords[0]]]
    df.rename(columns={"date": "ds", keywords[0]: "y"}, inplace=True)

    # Initialize and train Prophet model
    model = Prophet()
    model.fit(df)

    # Predict for next 3 months (12 weeks)
    future = model.make_future_dataframe(periods=12, freq='W')
    forecast = model.predict(future)

    # Interactive Trendline Plot
    fig = go.Figure()

    # Add actual trend data
    fig.add_trace(go.Scatter(x=df["ds"], y=df["y"], mode="lines", name="Actual Data", line=dict(color="blue")))

    # Add predicted trendline
    fig.add_trace(go.Scatter(x=forecast["ds"], y=forecast["yhat"], mode="lines", name="Predicted Trend", line=dict(color="red", dash="dot")))

    # Add confidence interval (upper & lower bounds)
    fig.add_trace(go.Scatter(x=forecast["ds"], y=forecast["yhat_upper"], mode="lines", name="Upper Bound", line=dict(color="lightgrey")))
    fig.add_trace(go.Scatter(x=forecast["ds"], y=forecast["yhat_lower"], mode="lines", name="Lower Bound", line=dict(color="lightgrey")))

    # Customize layout
    fig.update_layout(title=f"📈 Keyword Trend Prediction for {keywords[0]}",
                      xaxis_title="Date",
                      yaxis_title="Trend Score",
                      template="plotly_dark",
                      hovermode="x unified")

    # Show plot
    fig.show()

    # Show predicted values
    print(forecast[['ds', 'yhat']].tail(12))


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp67nan0t7/g0lw14ym.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp67nan0t7/xuaaib3k.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=80428', 'data', '

           ds       yhat
53 2025-03-30  65.575014
54 2025-04-06  65.184594
55 2025-04-13  64.794174
56 2025-04-20  64.403754
57 2025-04-27  64.013334
58 2025-05-04  63.622914
59 2025-05-11  63.232495
60 2025-05-18  62.842075
61 2025-05-25  62.451655
62 2025-06-01  62.061235
63 2025-06-08  61.670815
64 2025-06-15  61.280395


### Multi-Keyword Comparison

In [14]:
!pip install prophet plotly pandas pytrends

from pytrends.request import TrendReq
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import plotly.express as px

# Function to get Google Trends data dynamically
def get_trends(keywords, country="IN", timeframe="today 12-m"):
    pytrends = TrendReq(hl='en-US', tz=360)
    pytrends.build_payload(kw_list=keywords, timeframe=timeframe, geo=country)
    trends = pytrends.interest_over_time()

    if trends.empty:
        print("No data found. Try different parameters.")
    return trends

# === USER CUSTOMIZATIONS ===
keywords = ["Data Science", "Machine Learning", "SEO Automation"]  # Add any keywords
country = "IN"  # Change country ('US' for USA, 'GB' for UK, etc.)
timeframe = "today 12-m"  # Data from the past 12 months
forecast_days = 90  # Predict next 90 days (3 months)

# Get trends data
trends_data = get_trends(keywords, country, timeframe)

# Initialize the Plotly figure
fig = go.Figure()
colors = px.colors.qualitative.Set1  # Predefined color palette for clarity

# Process each keyword separately for forecasting
if not trends_data.empty:
    for idx, keyword in enumerate(keywords):
        df = trends_data.reset_index()[["date", keyword]]
        df.rename(columns={"date": "ds", keyword: "y"}, inplace=True)

        # Train Prophet model
        model = Prophet()
        model.fit(df)

        # Predict for next 'forecast_days' days (3 months)
        future = model.make_future_dataframe(periods=forecast_days, freq='D')  # Daily prediction
        forecast = model.predict(future)

        # Add actual trend data
        fig.add_trace(go.Scatter(
            x=df["ds"], y=df["y"], mode="lines", name=f"Actual - {keyword}",
            line=dict(color=colors[idx % len(colors)], width=3)
        ))

        # Add predicted trendline
        fig.add_trace(go.Scatter(
            x=forecast["ds"], y=forecast["yhat"], mode="lines",
            name=f"Predicted - {keyword}",
            line=dict(color=colors[idx % len(colors)], dash="dot", width=2)
        ))

        # Add confidence interval (transparent)
        fig.add_trace(go.Scatter(
            x=forecast["ds"].tolist() + forecast["ds"].tolist()[::-1],
            y=forecast["yhat_upper"].tolist() + forecast["yhat_lower"].tolist()[::-1],
            fill='toself',
            fillcolor=colors[idx % len(colors)].replace("rgb", "rgba").replace(")", ",0.2)"),  # Transparent fill
            line=dict(color="rgba(255,255,255,0)"),
            name=f"Confidence - {keyword}",
            showlegend=False
        ))

# Customize layout for clarity
fig.update_layout(
    title="📈 Customizable Multi-Keyword Trend Prediction (Next 3 Months)",
    xaxis_title="Date",
    yaxis_title="Trend Score",
    template="plotly_white",
    hovermode="x unified",
    margin=dict(l=40, r=40, t=40, b=40),
    legend=dict(x=0.01, y=1.05, orientation="h", bgcolor="rgba(255,255,255,0.7)")
)

# Show plot
fig.show()


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp67nan0t7/hsdd5scy.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp67nan0t7/ctp0gf6l.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=52684', 'data', 'file=/tmp/tmp67nan0t7/

## Auto-Detect Google Trends

In [21]:
!pip install prophet plotly pandas pytrends

from pytrends.request import TrendReq
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import plotly.express as px
import time
import random

# === USER CUSTOMIZATIONS ===
country = "US"  # Change to "IN" (India), "GB" (UK), etc.
timeframe = "today 12-m"  # Change timeframe ('today 3-m', 'today 6-m', 'today 5-y', etc.)
forecast_days = 90  # Predict next 90 days (3 months)
num_keywords = 5  # Number of trending keywords to detect
category = "0"  # Change category ("0" = all topics, "3" = tech, etc.)

# === FUNCTION TO GET TRENDING KEYWORDS SAFELY ===
def get_trending_keywords(country, num_keywords, category):
    pytrends = TrendReq(hl='en-US', tz=360)

    # Use related_queries() instead of trending_searches()
    keywords = ["AI", "Python", "SEO", "Machine Learning", "Data Science"]  # Default fallback
    try:
        pytrends.build_payload(kw_list=keywords, cat=category, timeframe="now 7-d", geo=country)
        related_queries = pytrends.related_queries()

        if related_queries:
            trending_keywords = []
            for queries in related_queries.values():
                if queries["top"] is not None:
                    trending_keywords.extend(queries["top"]["query"])

            return trending_keywords[:num_keywords]  # Get top trending keywords
    except Exception as e:
        print(f"⚠️ Error fetching trending keywords: {e}")

    return keywords  # Return fallback keywords if error occurs

# === FUNCTION TO GET GOOGLE TRENDS DATA ===
def get_trends(keywords, country, timeframe):
    pytrends = TrendReq(hl='en-US', tz=360)
    time.sleep(random.randint(5, 10))  # Random delay to prevent Google blocking

    try:
        pytrends.build_payload(kw_list=keywords, timeframe=timeframe, geo=country)
        trends = pytrends.interest_over_time()
        return trends if not trends.empty else None
    except Exception as e:
        print(f"⚠️ Error fetching trends: {e}")
        return None

# === AUTO-DETECT TRENDING KEYWORDS ===
keywords = get_trending_keywords(country, num_keywords, category)

if not keywords:
    print("⚠️ No keywords found. Please try again later.")
else:
    print(f"📌 Auto-detected trending keywords: {keywords}")

    # === FETCH TRENDS DATA ===
    trends_data = get_trends(keywords, country, timeframe)

    if trends_data is not None:
        # === VISUALIZATION SETUP ===
        fig = go.Figure()
        colors = px.colors.qualitative.Set1  # Color palette for clarity

        # === PROCESS EACH KEYWORD FOR FORECASTING ===
        for idx, keyword in enumerate(keywords):
            df = trends_data.reset_index()[["date", keyword]]
            df.rename(columns={"date": "ds", keyword: "y"}, inplace=True)

            # Train Prophet model
            model = Prophet()
            model.fit(df)

            # Predict for next 'forecast_days' days (3 months)
            future = model.make_future_dataframe(periods=forecast_days, freq='D')  # Daily prediction
            forecast = model.predict(future)

            # Add actual trend data
            fig.add_trace(go.Scatter(
                x=df["ds"], y=df["y"], mode="lines", name=f"Actual - {keyword}",
                line=dict(color=colors[idx % len(colors)], width=3)
            ))

            # Add predicted trendline
            fig.add_trace(go.Scatter(
                x=forecast["ds"], y=forecast["yhat"], mode="lines",
                name=f"Predicted - {keyword}",
                line=dict(color=colors[idx % len(colors)], dash="dot", width=2)
            ))

            # Add confidence interval (transparent)
            fig.add_trace(go.Scatter(
                x=forecast["ds"].tolist() + forecast["ds"].tolist()[::-1],
                y=forecast["yhat_upper"].tolist() + forecast["yhat_lower"].tolist()[::-1],
                fill='toself',
                fillcolor=colors[idx % len(colors)].replace("rgb", "rgba").replace(")", ",0.2)"),  # Transparent fill
                line=dict(color="rgba(255,255,255,0)"),
                name=f"Confidence - {keyword}",
                showlegend=False
            ))

        # === CUSTOMIZE LAYOUT ===
        fig.update_layout(
            title="📈 Auto-Detected Trending Keywords - 3 Month Prediction",
            xaxis_title="Date",
            yaxis_title="Trend Score",
            template="plotly_white",
            hovermode="x unified",
            margin=dict(l=40, r=40, t=40, b=40),
            legend=dict(x=0.01, y=1.05, orientation="h", bgcolor="rgba(255,255,255,0.7)")
        )

        # === SHOW PLOT ===
        fig.show()
    else:
        print("⚠️ No trend data available.")


⚠️ Error fetching trending keywords: list index out of range
📌 Auto-detected trending keywords: ['AI', 'Python', 'SEO', 'Machine Learning', 'Data Science']


/usr/local/lib/python3.11/dist-packages/pytrends/request.py:260: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp67nan0t7/90pap_lw.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp67nan0t7/8ysfmx9p.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=56008', 'data', 'file=/tmp/tmp67nan0t7/

**How to Customize? above auto predict model**

In [22]:
# Change Country
country = "US"  # USA
country = "IN"  # India
country = "GB"  # UK

# Change Timeframe
timeframe = "today 3-m"  # Last 3 months
timeframe = "today 6-m"  # Last 6 months
timeframe = "today 5-y"  # Last 5 years

# Change Forecast Duration
forecast_days = 90  # Next 3 months (default)
forecast_days = 180  # Next 6 months
forecast_days = 30  # Next 1 month

# Change Number of Auto-Detected Keywords
num_keywords = 5  # Detect top 5 trending keywords
num_keywords = 10  # Detect top 10 trending keywords

# Change Category
category = "0"  # All topics
category = "3"  # Technology
category = "19"  # Finance
category = "44"  # Health

#Full category list here- https://github.com/pat310/google-trends-api/wiki/Google-Trends-Categories
